In [2]:
import os
import cv2
import dlib
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import numpy as np

In [ ]:
import torch
import torch.transforms as transforms
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns

import tensorboardX

In [ ]:
epochs = 10
batch_size = 16

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [4]:
data_root = 'images/dataset'
save_path = 'images/image_dataset'

In [5]:
season_dict = {'spring':0, 'summer':1, 'fall':2, 'winter':3}

print(season_dict.keys())
print(season_dict.values())

dict_keys(['spring', 'summer', 'fall', 'winter'])
dict_values([0, 1, 2, 3])


In [6]:
num_files = 0

In [12]:
for season in season_dict:
    season_path = os.path.join(data_root, season)
    for file in os.listdir(season_path):
        full_path = os.path.join(season_path, file)
        img = cv2.imread(full_path)
        h, w = img.shape[:2]
        try:
            face = detector(img)[0]
        except:
            print(f'No face detected in {full_path}')
            os.remove(full_path)
            continue
        crop = img[max(0, face.top()):min(face.bottom(), h), max(0, face.left()):min(face.right(), w)]
        crop = cv2.resize(crop, (256, 256))
        cv2.imwrite(os.path.join(save_path, f'{season_dict[season]}_{num_files}.jpg'), crop)
        num_files += 1
    print(f'{num_files} images saved for {season}')
    num_files = 0
print(f'Total number of images saved: {len(os.listdir(save_path))}')

171 images saved for spring
No face detected in images/dataset\summer\119511005055_20071116.jpg
No face detected in images/dataset\summer\202303230812009144_0.jpg
No face detected in images/dataset\summer\220px-YuNaKimInVancouver.jpg
No face detected in images/dataset\summer\PYH2019060611910001300_P2.jpg
No face detected in images/dataset\summer\unnamed-file-186.jpg.webp
136 images saved for summer
No face detected in images/dataset\fall\1684360.jpg
No face detected in images/dataset\fall\l.yk..jpg
81 images saved for fall
No face detected in images/dataset\winter\gfsd-3.jpg
No face detected in images/dataset\winter\image_readtop_2022_764398_16618140035150682.jpg
193 images saved for winter
Total number of images saved: 581


In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
class FaceDataset(nn.Dataset):
    def __init__(self, n_classes, root_dir=save_path, transform=transform):
        self.root_dir = root_dir
        self.transform = transform
        self.files = os.listdir(self.root_dir)
        self.labels = [int(file.split('_')[0]) for file in self.files]
        
        self.n_classes = n_classes
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.files[idx])
        img = cv2.imread(img_name)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transform(img)
        label = self.labels[idx]
        return img, label

In [ ]:
dataset = FaceDataset(n_classes=4)
trainset, testset = train_test_split(dataset, test_size=0.2)
train_data = DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_data = DataLoader(testset, batch_size=batch_size, shuffle=True)

In [ ]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg16', pretrained=True)
model = model.to(device)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001).to(device)
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
writer = tensorboardX.SummaryWriter()

In [ ]:
def train(model, train_data, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, data in enumerate(train_data):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        writer.add_scalar('train_loss', loss.item(), i*epoch)
        writer.add_scalar('train_accuracy', 100*correct/total, i*epoch)
        
        if i % 10 == 99:
            print(f'[{i+1}, {running_loss/100:.3f}]')
            running_loss = 0.0
            correct = 0
            total = 0
    train_loss = running_loss/len(train_data)
    train_accuracy = 100*correct/total
    print(f'Training {epoch+1}/{epochs}: {train_loss:.3f}, Training Accuracy: {train_accuracy:.3f}')
    
    return train_loss, train_accuracy

In [ ]:
y_pred = []
y_true = []

def test(model, test_data, epoch):
    model.eval()
    
    correct = 0
    total = 0
    total_loss = 0
    
    with torch.no_grad():
        for batch_idx, data in enumerate(test_data):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            y_pred.extend(predicted.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
            
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            writer.add_scalar('test_accuracy', 100*correct/total, batch_idx*epoch)
            writer.add_scalar('test_loss', total_loss/(batch_idx+1), batch_idx*epoch)
    
    test_loss = total_loss/len(test_data)
    test_accuracy = 100*correct/total
    print(f'Testing {epoch+1}/{epochs}: {test_loss:.3f}, Testing Accuracy: {test_accuracy:.3f}')
    
    return test_loss, test_accuracy

In [ ]:
# epoch-wise losses and accuracies
# batch-wise losses and accuracies are logged in tensorboard
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

In [ ]:
for epoch in epochs:
    train_loss, train_acc = train(model, train_data, epoch)
    test_loss, test_acc = test(model, test_data, epoch)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)
    
    torch.save(model.state_dict(), f'models/face_model_{epoch}_{datetime.time}.pth')

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(test_accuracies, label='Testing Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Testing Accuracy')
plt.legend()
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(test_losses, label='Testing Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Testing Loss')
plt.legend()
plt.show()

In [ ]:
cm = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cm / np.sum(cm, axis=1)[:, None], index = [i for i in season_dict.keys()],
                     columns = [i for i in season_dict.keys()])
plt.figure(figsize = (12,7))
sns.heatmap(df_cm, annot=True)
plt.savefig('output.png')